In [2]:
import os
import shap
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras import regularizers
from imblearn.over_sampling import ADASYN

    
np.random.seed(42)


2023-04-20 20:10:55.161639: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
file_dir=('/Users/maddie/Projects/CPSC_597/')
#create text file to store results in and close again:
with open(file_dir+'/results/DNN_performance/DNN_eval_future.txt','w+') as file:
    file.write("Species"+"\t"+"Test_loss"+"\t"+"Test_acc"+"\t"+"Test_tpr"+"\t"+"Test_AUC"+"\t"+"Test_LCI95%"+"\t"+"Test_UCI95%"+"\t"+"occ_samples"+"\t"+"abs_samples"+"\n")
    file.close()
#access file with list of taxa names
taxa=pd.read_csv(file_dir+'/data/modified_data/gbif_filtered/taxa_list.txt',header=None)
taxa.columns=["taxon"] 

###column variable names
with open(file_dir+'data/data_raw/bio_oracle_future/variable_list.txt') as f:
      new_cols = f.readlines()


In [4]:
var_names=[]

for item in new_cols:
    item=item.replace("\n","")
    var_names.append(item) 

    
for species in taxa["taxon"][:]:
   
    #open dataframe and rename columns
    spec = species
    table = pd.read_csv(file_dir +"/data/modified_data/spec_ppa_future_env/%s_future_env_dataframe.csv"%spec)         
    table.rename(columns=dict(zip(table.columns[1:10], var_names)),inplace=True)
    
    ####################################
    #  filter dataframe for training   #
    ####################################
   
    # drop any row with no-data values
    table = table.dropna(axis=0, how="any")


    # make feature vector
    band_columns = [column for column in table.columns[1:10]]
    

    X = []
    y = []

    for _, row in table.iterrows():
        x = row[band_columns].values
        x = x.tolist()
        x.append(row["present/pseudo_absent"])
        X.append(x)

    df = pd.DataFrame(data=X, columns=band_columns + ["presence"])
    df.to_csv("filtered.csv", index=None)

    # extract n. of occ. and abs. samples
    occ_len=int(len(df[df["presence" ]==1]))
    abs_len=int(len(df[df["presence" ]==0]))
    
    ####################################
    #  Numpy feature and target array  #
    ####################################

    X = []
    y = []

    band_columns = [column for column in df.columns[:-1]]

    for _, row in df.iterrows():
        X.append(row[band_columns].values.tolist())
        y.append([1 - row["presence"], row["presence"]])

    X = np.vstack(X)
    y = np.vstack(y)

    ####################################
    #    Split training and test set   #
    ####################################

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y,random_state=42)
    
    test_set=pd.DataFrame(X_test)
    test_set.rename(columns=dict(zip(test_set.columns[0:9], var_names)),inplace=True)
    
    shuffled_X_train=X_train.copy()
    np.random.shuffle(shuffled_X_train)
    shuffled_X_train=shuffled_X_train[:1000] # random subsample from test set for feature importance
    
    shuffled_X_test=X_test.copy()
    np.random.shuffle(shuffled_X_test)
    shuffled_X_test=shuffled_X_test[:1000] # random subsample from test set for feature importance
    
    ####################################
    #      Training and testing        #
    ####################################
    
    # prepare metrics
    test_loss=[]
    test_acc=[]
    test_AUC=[]
    test_tpr=[]
    test_uci=[]
    test_lci=[]

   
    Best_model_AUC=[0]
    
    # Five repetitions
    for i in range(1,6):
        print("run %s"%i)
        ###################
        # Construct model #
        ###################
        batch_size = 75
        num_classes = 2
        epochs = 40

        num_inputs = X.shape[1]  # number of features


        model = Sequential()
        layer_1 = Dense(50, activation='relu',input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.000001))
        layer_2 = Dense(25, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.000001))
        layer_3 = Dense(25, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.0000001))
        layer_4 = Dense(25, activation='relu', input_shape=(num_inputs,))#, kernel_regularizer=regularizers.l1(0.00000001))


        model.add(layer_1)
        model.add(Dropout(0.3))
        model.add(layer_2)
        model.add(Dropout(0.5))
        model.add(layer_3)
        model.add(Dropout(0.3))
        model.add(layer_4)
        model.add(Dropout(0.5))


        out_layer = Dense(num_classes, activation=None)
        model.add(out_layer)
        model.add(Activation("softmax"))

        #model.summary()

        model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics =['accuracy'])
        
        ###############
        # Train model #
        ###############
        
        history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size, verbose=0)

        ##############
        # Test model #
        ##############
        score = model.evaluate(X_test, y_test, verbose=1)
        predictions = model.predict(X_test)
        fpr, tpr, thresholds = roc_curve(y_test[:, 1], predictions[:, 1])
        len_tpr=int(len(tpr)/2)
      

        #################
        # Append scores #
        #################
        test_loss.append(score[0])
        test_acc.append(score[1])
        test_AUC.append(roc_auc_score(y_test[:, 1], predictions[:, 1]))
        test_tpr.append(tpr[len_tpr])
        AUC = roc_auc_score(y_test[:, 1], predictions[:, 1])

        ###############################
        # Create confidence intervals #
        ###############################
        n_bootstraps=1000
        y_pred=predictions[:,1]
        y_true=y_test[:,1]
        rng_seed=42
        bootstrapped_scores =[]


        rng=np.random.RandomState(rng_seed)
        for i in range (n_bootstraps):
            #bootstrap by sampling with replacement on prediction indices
            indices = rng.randint(0,len(y_pred)-1,len(y_pred))
            if len (np.unique(y_true[indices])) <2:
                continue

            score = roc_auc_score(y_true[indices],y_pred[indices])
            bootstrapped_scores.append(score)

        sorted_scores=np.array(bootstrapped_scores)
        sorted_scores.sort()

        ci_lower=sorted_scores[int(0.05*len(sorted_scores))]
        ci_upper=sorted_scores[int(0.95*len(sorted_scores))]
     
        test_lci.append(ci_lower)
        test_uci.append(ci_upper)
       
    
        ##############################################################
        # Selection of best model across runs and feature importance #
        ##############################################################
    
        #determine whether new model AUC is higher
        if AUC > Best_model_AUC[0]:
            # if yes save model to disk / overwrite previous model
            Best_model_AUC[0]=AUC
            model_json=model.to_json()
            with open (file_dir+'results/fish_future/{}/{}_model.json'.format(spec,spec),'w') as json_file:
                json_file.write(model_json)
            model.save_weights(file_dir+'results/fish_future/{}/{}_model.h5'.format(spec,spec))
            #if yes, save a figure of shap feature value impact    
            
            if int(len(X_train)) > 5000:           
                explainer=shap.DeepExplainer(model,shuffled_X_train)
                test_set=pd.DataFrame(shuffled_X_test)
                test_set.rename(columns=dict(zip(test_set.columns[0:40], var_names)),inplace=True)
                
                shap_values=explainer.shap_values(shuffled_X_test)
                fig=shap.summary_plot(shap_values[1],test_set,show=False)
                plt.savefig(file_dir+'/results/fish_future/{}/{}_feature_impact'.format(spec,spec),bbox_inches="tight")
                plt.close()
            
            else:
                explainer=shap.DeepExplainer(model,X_train)
                shap_values=explainer.shap_values(X_test)
                fig=shap.summary_plot(shap_values[1],test_set,show=False)
                plt.savefig(file_dir+'/results/fish_future/{}/{}_feature_impact'.format(spec,spec),bbox_inches="tight")
                plt.close()
            


    # Model output metrics averaged across five runs to be written to file
    avg_loss= sum(test_loss)/len(test_loss)
    avg_acc = sum(test_acc)/len(test_acc)
    avg_AUC = sum(test_AUC)/len(test_AUC)
    avg_tpr = sum(test_tpr)/len(test_tpr)
    avg_lci = sum(test_lci)/len(test_lci)
    avg_uci = sum(test_uci)/len(test_uci)

    # Write to file
    with open(file_dir+'/results/DNN_performance/DNN_eval_future.txt','a') as file:
        file.write(spec+"\t"+str(avg_loss)+"\t"+str(avg_acc)+"\t"+str(avg_tpr)+"\t"+str(avg_AUC)+"\t"+str(avg_lci)+"\t"+str(avg_uci)+"\t"+str(occ_len)+"\t"+str(abs_len)+"\n")       


    #Next species!


run 1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                500       
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 dense_2 (Dense)             (None, 25)                650       
                                                                 
 dropout_2 (Dropout)         (None, 25)                0         
                                                                 
 dense_3 (Dense)             (None, 25)           

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 2
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 50)                500       
                                                                 
 dropout_4 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dropout_5 (Dropout)         (None, 25)                0         
                                                                 
 dense_7 (Dense)             (None, 25)                650       
                                                                 
 dropout_6 (Dropout)         (None, 25)                0         
                                                                 
 dense_8 (Dense)             (None, 25)         

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 2
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 50)                500       
                                                                 
 dropout_24 (Dropout)        (None, 50)                0         
                                                                 
 dense_31 (Dense)            (None, 25)                1275      
                                                                 
 dropout_25 (Dropout)        (None, 25)                0         
                                                                 
 dense_32 (Dense)            (None, 25)                650       
                                                                 
 dropout_26 (Dropout)        (None, 25)                0         
                                                                 
 dense_33 (Dense)            (None, 25)         

...

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 2
Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_105 (Dense)           (None, 50)                500       
                                                                 
 dropout_84 (Dropout)        (None, 50)                0         
                                                                 
 dense_106 (Dense)           (None, 25)                1275      
                                                                 
 dropout_85 (Dropout)        (None, 25)                0         
                                                                 
 dense_107 (Dense)           (None, 25)                650       
                                                                 
 dropout_86 (Dropout)        (None, 25)                0         
                                                                 
 dense_108 (Dense)           (None, 25)        

Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
`tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored


run 2
Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_130 (Dense)           (None, 50)                500       
                                                                 
 dropout_104 (Dropout)       (None, 50)                0         
                                                                 
 dense_131 (Dense)           (None, 25)                1275      
                                                                 
 dropout_105 (Dropout)       (None, 25)                0         
                                                                 
 dense_132 (Dense)           (None, 25)                650       
                                                                 
 dropout_106 (Dropout)       (None, 25)                0         
                                                                 
 dense_133 (Dense)           (None, 25)        

KeyboardInterrupt: 